#### sklearn.svm.SVC

* class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)[source]

* parameters:
    - C : float, default=1.0 ,c값이 클수록 복잡한 모형 대신 오버피팅이 되면서 margine이 작아짐
    - Gamma : {‘scale’, ‘auto’} or float, default=’scale’ , Gamma값이 클수록 복잡한 모형 클수록 서포트벡터로 잡는 벡터의 수가 작아짐
    - kernel : {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

In [1]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
x = iris.data
y = iris.target

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5)

clf = SVC(gamma=0.001,C=100)
clf.fit(x_train,y_train)
pred = clf.predict(x_test)
acc = accuracy_score(y_test,pred)

print(acc)

pram = {'C':[1,10,100],
       'gamma':[0.1,0.01,0.001]}
grid = GridSearchCV(clf,param_grid=pram,cv=3,scoring="accuracy")
grid.fit(x_train,y_train)
best_clf = grid.best_estimator_
best_pred = best_clf.predict(x_test)
best_acc = accuracy_score(y_test,best_pred)
print(best_acc)

0.9733333333333334
0.96


### human

In [7]:
import pandas as pd

feature_name_df = pd.read_csv('../ml/dataset/human_activity/features.txt',sep='\s+',
           header=None,names=['column_index','column_name'])
feature_name = feature_name_df.iloc[:,1].values.tolist()

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name'] =new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] >0 else x[0],axis=1)
    
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

def get_human_dataset():
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('../ml/dataset/human_activity/features.txt',sep='\s+',
           header=None,names=['column_index','column_name'])
    # 중복된 피처명을 수정하는 get_new_feature_name_df를 이용 신규 피쳐명 df생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DF에 피쳐명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터 셋을 DF로 로딩 컬러명은 feature_name적용
    x_train = pd.read_csv('../ml/dataset/human_activity/train/X_train.txt',sep="\s+",names=feature_name)
    x_test = pd.read_csv('../ml/dataset/human_activity/test/X_test.txt',sep="\s+",names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터를 DF로 할당 컬럼명은 action으로 부여
    y_train = pd.read_csv('../ml/dataset/human_activity/train/y_train.txt',sep="\s+",header=None,names=['action'])
    y_test = pd.read_csv('../ml/dataset/human_activity/test/y_test.txt',sep="\s+",header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 변환
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test=get_human_dataset()

In [11]:
### 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(random_state=156)
rf_clf.fit(x_train,y_train)
pred = rf_clf.predict(x_test)
accuracy = accuracy_score(y_test,pred)
print(f'랜덤포레스트 예측 정확도: {accuracy}')
print(f'기본 하이퍼파라미터 {rf_clf.get_params()}')


### gridsearchcv
from sklearn.model_selection import GridSearchCV

## 하이퍼파라미터 gridsearchcv
param = {'max_depth':[6,8,10,12],
        'n_estimators':[100],
        'min_samples_leaf':[8,12,18],
        'min_samples_split':[8,16,20]}

rf_clf = RandomForestClassifier(random_state=156,n_jobs=-1)
grid_cv = GridSearchCV(rf_clf,param_grid=param,cv=2,scoring='accuracy',n_jobs=-1)
grid_cv.fit(x_train,y_train)
best_dt = grid_cv.best_estimator_

### 변수선택
ftf_importances_values = best_dt.feature_importances_
importances = pd.Series(ftf_importances_values,index=x_train.columns)
top10 = importances.sort_values(ascending=False)[:10]

랜덤포레스트 예측 정확도: 0.9236511706820495
기본 하이퍼파라미터 {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 156, 'verbose': 0, 'warm_start': False}


In [12]:
top10_index = top10.index
x_train_10 = x_train.loc[:,top10_index]
x_test_10 = x_test.loc[:,top10_index]


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


clf = SVC(gamma=0.001,C=100)
clf.fit(x_train_10,y_train)
pred = clf.predict(x_test_10)
acc = accuracy_score(y_test,pred)

print(acc)

pram = {'C':[1,10,100],
       'gamma':[0.1,0.01,0.001]}
grid = GridSearchCV(clf,param_grid=pram,cv=2,scoring="accuracy")
grid.fit(x_train_10,y_train)
best_clf = grid.best_estimator_
best_pred = best_clf.predict(x_test_10)
best_acc = accuracy_score(y_test,best_pred)

print(best_acc)

0.5225653206650831
0.6386155412283678
